In [7]:
import cv2
from ipywidgets import Video
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [8]:
video_path = 'Chess_Data\\chess_easy1.mp4'

In [9]:
video_capture = cv2.VideoCapture(video_path)

### Read one frame to calibrate camera

In [10]:
def save_first_frame(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Release the video capture object
    cap.release()

    return frame

first_frame = save_first_frame(video_path)

In [11]:
def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

In [12]:
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(gray, 50, 150)

frame_with_boxes = first_frame.copy()
dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
dst = cv2.dilate(dst, None)
corners = np.argwhere(dst > 0.01 * dst.max())

# Find corners closest to image corners
image_corners = np.array([[0, 0], [0, first_frame.shape[1]], [first_frame.shape[0], 0], [first_frame.shape[0], first_frame.shape[1]]])
closest_corners = find_closest_points_to_corners(corners, image_corners)

target_square_size = 1000  # Adjust the size as needed
target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)

### Full detection

In [13]:
video_capture = cv2.VideoCapture(video_path)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    
    homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)

    # Apply the perspective transformation
    calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

    # Display the result
    cv2.imshow('Chessboard focus', calibrated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()